In [60]:
import gns3fy
from gns3fy import Gns3Connector
from gns3fy import Node, Link

In [61]:
server = Gns3Connector('http://localhost:3080')
lab = gns3fy.Project(name='test', connector=server)
lab.get()

In [62]:
pid = lab.project_id

In [63]:
lab.status

'opened'

In [64]:
lab.links_summary()

alpine1: Ethernet0 ---- alpine2: Ethernet0
alpine1: Ethernet1 ---- host1: Ethernet0
alpine1: Ethernet2 ---- host2: Ethernet0


In [65]:
host1 = Node(project_id=pid, name='host1', connector=server, template='VPCS')
host2 = Node(project_id=pid, name='host2', connector=server, template='VPCS')

In [7]:
# host1.create()
# host2.create()

host1.get()
host2.get()

In [8]:
host1.ports

[{'adapter_number': 0,
  'data_link_types': {'Ethernet': 'DLT_EN10MB'},
  'link_type': 'ethernet',
  'name': 'Ethernet0',
  'port_number': 0,
  'short_name': 'e0'}]

In [9]:
# nodes = [
#     dict(node_id=alpine1.node_id, adapter_number=0, port_number=1),
#     dict(node_id=host1.node_id, adapter_number=0, port_number=0)
# ]

# h1_ap1_link = Link(project_id=pid, nodes=nodes, connector=server)
# h1_ap1_link.create()

In [10]:
# nodes = [
#     dict(node_id=alpine1.node_id, adapter_number=0, port_number=2),
#     dict(node_id=host2.node_id, adapter_number=0, port_number=0)
# ]

# h2_ap1_link = Link(project_id=pid, nodes=nodes, connector=server)
# h2_ap1_link.create()

In [27]:
import telnetlib
import time

In [86]:
h1_console = telnetlib.Telnet('127.0.0.1', 5002, timeout=30)
h1_console.write(b'show ip\n')
data = h1_console.read_until(b'EOF\n', timeout=0.1)
f = open('ips', 'w')
f.write(data.decode('utf-8'))
f.close()
h1_console.close()

In [106]:
import re
pattern = r'(\d+.\d+.\d+.\d+/\d+)'
match = re.search(pattern, data.decode("utf-8"))
if match:
        print(match.group(1))

192.168.1.2/8


In [104]:
import re
def extract_ip(data):        
    pattern = r'(\d+.\d+.\d+.\d+/\d+)'
    match = re.search(pattern, data.decode("utf-8"))
    if match:
        return match.group(1)
    raise KeyError()

In [107]:
def get_ip(node) -> None:
    console = telnetlib.Telnet('127.0.0.1', node.console, timeout=5)
    console.write(b'show ip\n')
    data = console.read_until(b'EOF\n', timeout=0.1)
    try:
        ip = extract_ip(data)
    except KeyError:
        print(f'ip not found for node {node.name} with port: {node.console}')

    f = open('ips', 'a')
    f.write(f'{node.name}, {node.console}, {ip}\n')
    f.close()
    console.close()


In [108]:
nodes = lab.nodes
for node in nodes:
    if node.node_type == 'vpcs':
        get_ip(node)

##### aight now to build the tree topology : )
##### soooo how would it look like sef

In [ ]:
def build_topo():
    